# Failed models

Some reconstructions could not be parsed into solving models. This is attributed to errors in encoding or decoding the reconstruction file.

However, [cobrapy](https://opencobra.github.io/cobrapy) finds no growth for these models with both floating point and exact solvers.

In [1]:
import os
import re

import scipy.io
import pandas

import cobra

from read_excel import read_excel

In [2]:
def open_exchanges(model, amount=10):
    for reaction in model.reactions:
        if len(reaction.metabolites) == 1:
            # Ensure we are not creating any new sinks
            if reaction.metabolites.values()[0] > 0:
                reaction.upper_bound = max(reaction.upper_bound, amount)
            else:
                reaction.lower_bound = min(reaction.lower_bound, -amount)

def add_exchanges(model, extracellular_suffix="[e]", uptake_amount=10):
    for metabolite in model.metabolites:
        if str(metabolite).endswith(extracellular_suffix):
            if len(metabolite.reactions) == 0:
                print "no reactions for " + metabolite.id
                continue
            if min(len(i.metabolites) for i in metabolite.reactions) > 1:
                EX_reaction = cobra.Reaction("EX_" + metabolite.id)
                EX_reaction.add_metabolites({metabolite: 1})
                m.add_reaction(EX_reaction)
                EX_reaction.upper_bound = uptake_amount
                EX_reaction.lower_bound = -uptake_amount

In [3]:
models = cobra.DictList()

In [4]:
m = read_excel("failed/iOA584.xls", verbose=False)
add_exchanges(m, "xt")
models.append(m)

In [5]:
m = read_excel("failed/iGT196.xls", verbose=False)
add_exchanges(m)
models.append(m)

In [6]:
m = read_excel("failed/iKK446.xls", rxn_sheet_name="List of Reactions", rxn_sheet_header=1,
               rxn_id_key="Gene name", rxn_str_key="Reactions", rxn_gpr_key="ORF", verbose=False)
add_exchanges(m, "xt")
models.append(m)

In [7]:
m = read_excel("failed/iYS277.xls", rxn_id_key="auto", rxn_str_key="Unnamed: 3", rxn_gpr_key="gene name",
               # a metabolite "fake" is added to the beginning of each string to prevent the compartments
               # at the beginnning from being applied to the whole thing
               rxn_sheet_converters={"Unnamed: 3": lambda x: "fake + " + x if len(x.strip()) > 0 else ""}, verbose=False)
for met in m.metabolites:
    if met.id.startswith("[e]"):
        met.id = met.id[3:] + "[e]"
m.repair()
m.metabolites.fake.remove_from_model()
add_exchanges(m, "[e]")
models.append(m)

cobra/core/Reaction.py:98 UserWarning: malformed gene_reaction_rule 'NCgl2521??????????' for <Reaction R0017 at 0x7f24cf10d150>


In [8]:
for i in os.listdir("failed"):
    if not i.endswith(".xml"):
        continue
    model = cobra.io.read_legacy_sbml(os.path.join("failed", i))
    model.id = i[:-4]
    models.append(model)

## Autodetect objectives

In [9]:
biomass_re = re.compile("biomass", re.IGNORECASE)
curated_objectives = {"iOA584": "rxn1387", "iIN800": "GROWTH", "iGT196": "VGRO", "iMH551": "R0227"}

In [10]:
for m in models:
    if len(m.reactions.query(lambda x: x > 0, "objective_coefficient")):
        continue
    if m.id in curated_objectives:
        m.change_objective(curated_objectives[m.id])
        continue
    
    # look for reactions with "biomass" in the id or name
    possible_objectives = m.reactions.query(biomass_re)
    if len(possible_objectives) == 0:
        possible_objectives = m.reactions.query(biomass_re, "name")
    
    # In some cases, a biomass "metabolite" is produced, whose production
    # should be the objective function.
    possible_biomass_metabolites = m.metabolites.query(biomass_re)
    if len(possible_biomass_metabolites) == 0:
        possible_biomass_metabolites = m.metabolites.query(biomass_re, "name")

    if len(possible_biomass_metabolites) > 0:
        biomass_met = possible_biomass_metabolites[0]
        r = cobra.Reaction("added_biomass_sink")
        r.objective_coefficient = 1
        r.add_metabolites({biomass_met: -1})
        m.add_reaction(r)
        print ("autodetected biomass metabolite '%s' for model '%s'" %
              (biomass_met.id, m.id))

    elif len(possible_objectives) > 0:
        print("autodetected objective reaction '%s' for model '%s'" %
              (possible_objectives[0].id, m.id))
        m.change_objective(possible_objectives[0])

    
    else:
        print("no objective found for " + m.id)
        models.remove(m.id)

# Ensure the biomass objective flux is unconstrained
for reaction in m.reactions.query(lambda x: x > 0, "objective_coefficient"):
    reaction.lower_bound = min(reaction.lower_bound, 0)
    reaction.upper_bound = max(reaction.upper_bound, 1000)

autodetected biomass metabolite 'BIOMASS' for model 'iKK446'
autodetected biomass metabolite 'Biomass' for model 'iYS277'
autodetected biomass metabolite 'Biomass_c' for model 'PlasmoNet'
no objective found for iMH805


Maybe some metabolites need to be merged

In [11]:
def merge_metabolites(m, met_id1, met_id2):
    met1 = m.metabolites.get_by_id(met_id1)
    met2 = m.metabolites.get_by_id(met_id2)
    for reaction in list(met2.reactions):
        reaction.add_metabolites({met1: reaction.metabolites[met2]})
    met2.remove_from_model()

merge_metabolites(models.iOA584, "gln-L", "glnL")

## Solve

In [12]:
results = {}
for m in models:
    # solve this model with all the solvers
    solutions = {solver: m.optimize(solver=solver)
                 for solver in cobra.solvers.solver_dict}
    solutions["cglpk_exact"] = m.optimize(solver="cglpk", exact=True)
    # store the objective value and errors
    results[m.id] = {k: v.f for k, v in solutions.iteritems()}

results = pandas.DataFrame.from_dict(results, orient='index')
results

,mosek,esolver,cglpk,cplex,gurobi,cglpk_exact,coin,glpk
PlasmoNet,0,0,0.000000e+00,0.000000e+00,-0,0,-1.140600e-13,0.000000e+00
iGT196,0,0,0.000000e+00,0.000000e+00,-0,0,7.506453e-08,0.000000e+00
iIN800,0,0,4.144976e-14,1.136868e-13,-0,0,2.116824e-11,3.889961e-15
iKK446,0,0,0.000000e+00,0.000000e+00,-0,0,5.872481e-10,6.705318e-29
iOA584,0,0,0.000000e+00,0.000000e+00,-0,0,7.911445e-10,0.000000e+00
iYS277,0,0,0.000000e+00,0.000000e+00,-0,0,0.000000e+00,0.000000e+00


All the growth rates found are below 1e-6, which is consistent with esolver.

In [13]:
abs(results).max().max()

7.5064531298271003e-08